In [ ]:
import scipy.io
import numpy as np
import pandas as pd
import scanpy as sc
import h5py

import os
import numpy as np
import pandas as pd
import scanpy as sc
import anndata 
import seaborn as sns
from scipy.stats import zscore
import matplotlib.pyplot as plt
import collections
from natsort import natsorted

from scipy import stats
from scipy import sparse
from sklearn.decomposition import PCA
from umap import UMAP

from matplotlib.colors import LinearSegmentedColormap
from statsmodels.stats.multitest import multipletests

from scroutines.config_plots import *
from scroutines import powerplots # .config_plots import *
from scroutines import pnmf
from scroutines import basicu

In [ ]:
def split_peakstring(peak):
    """
    """
    a, bc = peak.split(':')
    b, c = bc.split('-')
    b, c = int(b), int(c)
    return a,b,c

def peaksstrings_to_bed(peaks, name=None, fout=None, **kwargs):
    """
    """
    
    bedarray = np.char.replace(np.char.replace(peaks, ':', '\t'), '-', '\t')
    if name is not None:
        lastcol = np.arange(len(peaks)).astype(str)
        linker = f"\t{name}_"
        bedarray = np.char.add(np.char.add(bedarray, linker), lastcol) 
        
    if fout is not None:
        print(f"saving to {fout}")
        np.savetxt(fout, bedarray, **kwargs)
        
    return bedarray

# [(a,b,c) for a,b,c in map(split_peakstring, a_peaks[:10])]
# peaksstrings_to_bed(a_peaks, name='P6_A')


# rna results

In [ ]:
import importlib
import scroutines
from scroutines.gene_modules import GeneModules
importlib.reload(scroutines)
importlib.reload(scroutines.gene_modules)


gene_modules = GeneModules()
g, gs, ms = gene_modules.check_genes('Cdh13')
print("\t".join(g))
print("\t".join(gs))
print("\t".join(ms))

In [ ]:
%%time
f_rna0 = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/L23_allmultiome_proc_P6toP21.h5ad'
f_rna1 = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/rna_qs_avc_p6to21.txt'
f_rna2 = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/rna_l2fc_avc_p6to21.txt'

adata_rna = sc.read(f_rna0, backed='r')
rna_qs_avc = np.loadtxt(f_rna1)
rna_l2fc_avc = np.loadtxt(f_rna2)

rna_qs_avc.shape, rna_l2fc_avc.shape

In [ ]:
ts = [6,8,10,12,14,17,21]

In [ ]:
genes = adata_rna.var.index.values
degs_a = []
degs_c = []
for i, t in enumerate(ts):
    cond_a = np.logical_and(rna_qs_avc[i]<0.05, rna_l2fc_avc[i] < -1)
    cond_c = np.logical_and(rna_qs_avc[i]<0.05, rna_l2fc_avc[i] >  1)
    deg_a = np.sort(genes[cond_a])
    deg_c = np.sort(genes[cond_c])
    degs_a.append(deg_a)
    degs_c.append(deg_c)
    
    print(i, f"P{t}")
    g, gs, ms = gene_modules.check_genes(deg_a)
    print("\t".join(gs))
    print("\033[0;m ---")
    g, gs, ms = gene_modules.check_genes(deg_c)
    print("\t".join(gs))
    print("\033[0;m ---")
    

# atac results

In [ ]:
din = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results_atac'
!ls $din

In [ ]:
stats = []
for i, t in enumerate(ts):
    fin = din + f'/P{t}_A_peaks_ngene.bed'
    df_a = pd.read_csv(fin, sep='\t', header=None)
    fin = din + f'/P{t}_C_peaks_ngene.bed'
    df_c = pd.read_csv(fin, sep='\t', header=None)
    dag_a = df_a[7].unique()
    dag_c = df_c[7].unique()
    dag_union = np.union1d(dag_a, dag_c)

    deg_a = degs_a[i]
    deg_c = degs_c[i]
    print(i,t)
    
    num_a  = len(deg_a)
    num_aa = len(np.intersect1d(deg_a, dag_a))
    num_ac = len(np.intersect1d(deg_a, dag_c))
    num_an = len([a for a in deg_a if a not in dag_union])
    
    num_c  = len(deg_c)
    num_cc = len(np.intersect1d(deg_c, dag_c))
    num_ca = len(np.intersect1d(deg_c, dag_a))
    num_cn = len([c for c in deg_c if c not in dag_union])
    
    print("A: a,aa,ac,an ", num_a, num_aa, num_ac, num_an)
    print("C: c,cc,ca,cn ", num_c, num_cc, num_ca, num_cn)
    
    stats.append({'a':  num_a,
                  'aa': num_aa,
                  'ac': num_ac,
                  'an': num_an,
                  
                  'c':  num_c,
                  'cc': num_cc,
                  'ca': num_ca,
                  'cn': num_cn,
                 })
    
stats = pd.DataFrame(stats)
stats
    
    # break

In [ ]:
sns.color_palette('tab20c')

In [ ]:
C0v2 = sns.color_palette('tab20c')[2]
C1v2 = sns.color_palette('tab20c')[2+8]

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(2*5,1*5), sharey=True, sharex=True)
ax = axs[0]
ax.set_title('A genes')
ax.set_ylim(0,1)
ax.plot(ts, stats['aa']/stats['a'],  '-o', label='near A peaks', color='C0')
ax.plot(ts, stats['ac']/stats['a'],  '-o', label='near C peaks', color=C0v2)
ax.plot(ts, stats['an']/stats['a'],  '-o', label='neither', color='lightgray')
ax.set_xticks(ts)
ax.grid(axis='x')
sns.despine(ax=ax)
ax.set_ylabel("frac. of genes")
ax.set_xlabel("time (P)")
ax.legend(bbox_to_anchor=(0.5,-0.2), loc='upper center')


ax = axs[1]
ax.set_title('C genes')
ax.plot(ts, stats['cc']/stats['c'],  '-o', label='near C peaks', color='C2')
ax.plot(ts, stats['ca']/stats['c'],  '-o', label='near A peaks', color=C1v2)
ax.plot(ts, stats['cn']/stats['c'],  '-o', label='neither', color='lightgray')
ax.set_xticks(ts)
ax.grid(axis='x')
sns.despine(ax=ax)
ax.set_ylabel("frac. of genes")
ax.set_xlabel("time (P)")
ax.legend(bbox_to_anchor=(0.5,-0.2), loc='upper center')

plt.show()

# gene annot

In [ ]:
a_all_annots, a_all_styled, a_all_annots_styled = gene_modules.check_genes(a_all)
c_all_annots, c_all_styled, c_all_annots_styled = gene_modules.check_genes(c_all)

print("\t".join(a_all))
print("\t".join(a_all_annots_styled)) # _styled))

In [ ]:
print("\t".join(c_all))
print("\t".join(c_all_annots_styled)) # _styled))

In [ ]:
print("\033[0;m alltime A:", "\t".join(a_all_styled))
print("\033[0;m alltime C:", "\t".join(c_all_styled))

# Volcano

In [ ]:
def show_volcano_v2(thetypeidx, thetype, lfc, qs,
                    cond1, cond2up, cond2dn, 
                    querygenes_idx=None, 
                    gene_annots=None,
                    ax=None, bbox_to_anchor=(1,1), loc=None,
                   ): 
    """
    """
    eff = lfc[:,thetypeidx]
    pvl = -np.log10(qs[:,thetypeidx]) # +1e-10)
    cnd_up = np.all([cond1[:,thetypeidx], 
                     cond2up[:,thetypeidx]], axis=0) 
    cnd_dn = np.all([cond1[:,thetypeidx], 
                     cond2dn[:,thetypeidx]], axis=0) 

    if ax is None: 
        fig, ax = plt.subplots()
    
    # all genes
    ax.scatter(eff, pvl, s=1, color='lightgray', rasterized=True)

    # up genes
    ax.scatter(eff[cnd_up], pvl[cnd_up], s=3, facecolors='C0', rasterized=True)
    # dn genes
    ax.scatter(eff[cnd_dn], pvl[cnd_dn], s=3, facecolors='C1', rasterized=True)
    
    # # query genes
    # add text
    if querygenes_idx is not None:
        ax.scatter(eff[querygenes_idx], pvl[querygenes_idx], s=15, 
                   # label=f'type-specific (n={len(querygenes_idx):,})',
                   facecolors='none', edgecolors='k', linewidth=1, rasterized=True)
        for idx in querygenes_idx:
            ax.text(eff[idx], pvl[idx], gene_annots[idx], fontsize=10)

            
    # ax.grid(axis='y')
    sns.despine(ax=ax)
    ax.set_xlabel('log2(FC) (C/A in CP10k)')
    ax.set_ylabel('-log10(adj. p)')
    ax.set_title(f'{thetype}')
    ax.text(1,0.1,
            f'up (n={cnd_up.sum():,})\ndown (n={cnd_dn.sum():,})', 
            ha='right',
            fontsize=10, transform=ax.transAxes)
    return 

In [ ]:
unq_condidx = adata.obs['cond_order'].unique()
unq_condidx, unq_conds

In [ ]:
genes_comm = adata.var.index.values
lfc = adata.uns['l2fc_avc'].T
qs  = adata.uns['qs_avc'].T
lfc_th, qs_th = 1, 0.05

In [ ]:
cond1   = qs  <  qs_th
cond2up = lfc >  lfc_th
cond2dn = lfc < -lfc_th

In [ ]:
# typegenes_idx = basicu.get_index_from_array(genes_comm, df2.gene)

In [ ]:
thetypeidx = 0 # type A 
thetype = 'L2/3_A'
show_volcano_v2(thetypeidx, thetype, lfc, qs, cond1, cond2up, cond2dn) #  typegenes_idx)

In [ ]:
n = len(unq_conds)
fig, axs = plt.subplots(1,n,figsize=(4*n,4*1), sharex=True, sharey=True)
for cond_idx in unq_condidx:
    ax = axs.flat[cond_idx]
    thecond = unq_conds[cond_idx]
    show_volcano_v2(cond_idx, thecond, lfc, qs, cond1, cond2up, cond2dn, 
                    # typegenes_idx, 
                    ax=ax, bbox_to_anchor=(0.5, -0.3), loc='upper center')
    sns.despine(ax=ax)
fig.tight_layout()

# output = os.path.join(outfigdir, "volcano.pdf")
# powerplots.savefig_autodate(fig, output)
plt.show()

In [ ]:
querygenes = ['Meis2','Foxp1','Cdh13','Cdh12']
querygenes_idx = basicu.get_index_from_array(genes_comm, querygenes) 
gene_annots = genes_comm

n = len(unq_conds)
fig, axs = plt.subplots(2,4,figsize=(4*4,4*2), sharex=True, sharey=True)
for cond_idx in unq_condidx:
    ax = axs.flat[cond_idx]
    thecond = unq_conds[cond_idx]
    show_volcano_v2(cond_idx, thecond, lfc, qs, cond1, cond2up, cond2dn, 
                    querygenes_idx=querygenes_idx, 
                    gene_annots=genes_comm,
                    ax=ax, bbox_to_anchor=(0.5, -0.3), loc='upper center')
    sns.despine(ax=ax)
fig.tight_layout()
plt.show()

In [ ]:
# all tfs 
# not a meaningful test

In [ ]:
genes_sig = genes_comm[np.any(np.abs(lfc) > 1, axis=1)]
querygenes = np.intersect1d(annots['tf'], genes_sig) #['Meis2','Foxp1','Cdh13','Cdh12']
querygenes_idx = basicu.get_index_from_array(genes_comm, querygenes) 
gene_annots = genes_comm

n = len(unq_conds)
fig, axs = plt.subplots(2,4,figsize=(4*4,4*2), sharex=True, sharey=True)
for cond_idx in unq_condidx:
    ax = axs.flat[cond_idx]
    thecond = unq_conds[cond_idx]
    show_volcano_v2(cond_idx, thecond, lfc, qs, cond1, cond2up, cond2dn, 
                    querygenes_idx=querygenes_idx, 
                    gene_annots=genes_comm,
                    ax=ax, bbox_to_anchor=(0.5, -0.3), loc='upper center')
    sns.despine(ax=ax)
fig.tight_layout()
plt.show()